In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

# Data preparation

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv'
df = pd.read_csv(data)
df.columns = df.columns.str.lower()

status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)

home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

df = df[df.status != 'unk'].reset_index(drop=True)

In [3]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_test['status']
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

## Random forest

In [4]:

rf = RandomForestClassifier(n_estimators=200,
                            max_depth=10,
                            min_samples_leaf=3,
                            random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, n_estimators=200,
                       random_state=1)

## XGBoost

In [5]:
dtrain = xgb.DMatrix(X_train, label=y_train)
xgb_params = {
    'eta': 0.1, 
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=175)

In [9]:
train_dicts

[{'seniority': 22,
  'home': 'owner',
  'time': 48,
  'age': 48,
  'marital': 'married',
  'records': 'no',
  'job': 'fixed',
  'expenses': 60,
  'income': 110.0,
  'assets': 3000.0,
  'debt': 0.0,
  'amount': 1000,
  'price': 1460},
 {'seniority': 8,
  'home': 'other',
  'time': 60,
  'age': 41,
  'marital': 'married',
  'records': 'no',
  'job': 'freelance',
  'expenses': 45,
  'income': 62.0,
  'assets': 0.0,
  'debt': 0.0,
  'amount': 1800,
  'price': 2101},
 {'seniority': 2,
  'home': 'parents',
  'time': 36,
  'age': 19,
  'marital': 'married',
  'records': 'no',
  'job': 'fixed',
  'expenses': 35,
  'income': 162.0,
  'assets': 4000.0,
  'debt': 100.0,
  'amount': 400,
  'price': 570},
 {'seniority': 1,
  'home': 'owner',
  'time': 12,
  'age': 61,
  'marital': 'married',
  'records': 'no',
  'job': 'others',
  'expenses': 45,
  'income': 103.0,
  'assets': 20000.0,
  'debt': 0.0,
  'amount': 300,
  'price': 650},
 {'seniority': 13,
  'home': 'owner',
  'time': 60,
  'age': 27,


# 7.0 BENTOML

In [6]:
import bentoml


In [7]:
bentoml.xgboost.save_model('credit_risk_model',model,
                          custom_objects = {
                              "dictVectorizer":dv
                          })

Model(tag="credit_risk_model:r2iqmy2sr2l5jshc", path="C:\Users\eddye\bentoml\models\credit_risk_model\r2iqmy2sr2l5jshc\")